In [ ]:
!pip uninstall -y -q torch torchvision sympy
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121 # Replace cu118 with your CUDA version or remove if using CPU
!pip install -q sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [ ]:
from tqdm.notebook import tqdm # Import tqdm
import requests # Added for downloading
import urllib.parse # Added for URL encoding category names
import torch
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import time
import struct # For unpacking binary data
from struct import unpack
import os
import collections
import logging
from datetime import datetime
import csv
# import urllib.request # Removed: No longer downloading
from PIL import Image, ImageDraw


In [ ]:
# Configuration
QUICKDRAW_CATEGORIES = [
    'apple', 'cat', 'dog', 'door', 'elephant', 'fish', 'flower', 'table',
    'grass', 'house', 'ice cream', 'circle', 'key', 'lion', 'moon', 'nose',
    'pencil', 'rabbit', 'sun', 'tree', 'umbrella', 'van', 'cake', 'airplane',
    'ant', 'banana', 'bed', 'bee', 'bicycle', 'bird', 'book', 'bread', 'bus',
    'elbow', 'ear', 'camera', 'car', 'chair', 'clock', 'cloud', 'hand',
    'computer', 'cookie', 'cow', 'crayon', 'cup', 'eraser', 'carrot', 'drums',
    'eye', 'knife'
]
# NUM_TRAIN_SAMPLES_PER_CATEGORY = 5000
NUM_TRAIN_SAMPLES_PER_CATEGORY = 100
# NUM_TEST_SAMPLES_PER_CATEGORY = 1000
NUM_TEST_SAMPLES_PER_CATEGORY = 20
QUICKDRAW_CACHE_SIZE=20000
IMAGE_SIZE = (256, 256)
LINE_WIDTH = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BINARY_DATA_ROOT = './data' # MODIFIED: Point to local data directory

# Configuration section - add dynamic batch size scaling
BATCH_SIZE = 64  # Base batch size for smaller datasets
# Add adaptive batch size based on dataset size
MAX_BATCH_SIZE = 64  # Maximum batch size to prevent OOM errors
MIN_BATCH_SIZE = 16  # Minimum batch size for efficiency

# Add a function to calculate appropriate batch size
def get_adaptive_batch_size(num_samples_per_category, num_categories):
    """Calculate appropriate batch size based on dataset size to prevent OOM errors"""
    total_samples = num_samples_per_category * num_categories
    
    if total_samples < 10000:  # Small dataset
        return MAX_BATCH_SIZE
    elif total_samples < 50000:  # Medium dataset
        return 32
    elif total_samples < 200000:  # Large dataset
        return 16
    else:  # Very large dataset
        return MIN_BATCH_SIZE
    
# Fine-tuning hyperparameters - updated for better training
NUM_FINETUNE_EPOCHS = 10               # Increased from 3 to 10 epochs
FINETUNE_LEARNING_RATE = 5e-4          # Slightly increased from 1e-4
FINETUNE_WEIGHT_DECAY = 1e-5           # Added weight decay for regularization
MODEL_SAVE_PATH = './models'                # Directory to save fine-tuned models
VALIDATION_SPLIT = 0.1                 # Percentage of training data to use for validation
USE_GRADUAL_UNFREEZING = True          # Whether to use gradual unfreezing
USE_DATA_AUGMENTATION = True           # Whether to use data augmentation
GRADIENT_ACCUMULATION_STEPS = 1  # Default: update weights after every batch


# --- Model Definitions and Feature Extractors (Unchanged) ---
MODELS_TO_TEST = {
    "MobileNetV3-Small": {
        "weights": models.MobileNet_V3_Small_Weights.IMAGENET1K_V1,
        "model_fn": models.mobilenet_v3_small,
        "feature_extractor_fn": lambda m: MobileNetV3FeatureExtractor(m)
    },
    # "ShuffleNetV2_x0_5": {
    #     "weights": models.ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1,
    #     "model_fn": models.shufflenet_v2_x0_5,
    #     "feature_extractor_fn": lambda m: ShuffleNetV2FeatureExtractor(m)
    # },
    "SqueezeNet1_1": {
        "weights": models.SqueezeNet1_1_Weights.IMAGENET1K_V1,
        "model_fn": models.squeezenet1_1,
        "feature_extractor_fn": lambda m: SqueezeNetFeatureExtractor(m)
    },
    # "EfficientNet-B0": {
    #     "weights": models.EfficientNet_B0_Weights.IMAGENET1K_V1,
    #     "model_fn": models.efficientnet_b0,
    #     "feature_extractor_fn": lambda m: EfficientNetFeatureExtractor(m)
    # }
}

# --- Logging Configuration ---
# Set to logging.DEBUG for verbose development output, logging.INFO for less
LOG_LEVEL = logging.INFO

# Create a logs directory if it doesn't exist
LOGS_DIR = './logs'
if not os.path.exists(LOGS_DIR):
    os.makedirs(LOGS_DIR, exist_ok=True)

# Generate a log file name based on the notebook file name, datetime, and environment
notebook_name = 'quickdraw_benchmark'
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
log_file_name = f"{notebook_name}_{timestamp}_{device_type}.log"
log_file_path = os.path.join(LOGS_DIR, log_file_name)

# Get the root logger (or a specific logger, __name__ is fine)
# Avoid using basicConfig if you need fine-grained handler control after creation
# basicConfig configures the root logger, but we can get it and clear handlers
# or just create our own logger from scratch. Let's create our own logger explicitly.
logger = logging.getLogger(__name__)
logger.setLevel(LOG_LEVEL) # Set the level for *this* logger

# Prevent duplicate handlers if the cell is run multiple times
if logger.hasHandlers():
    logger.handlers.clear()

# Create handlers manually
# File Handler: Use buffering=1 for line buffering (most common for text) or 0 for no buffering
# For binary data like the .bin files are processed from, default buffering applies.
# However, the FileHandler *itself* writes text logs, so we can try line buffering.
# If that's not sufficient, we could force flush periodically.
try:
    # Using buffering=1 for line buffering in text mode ('w') is standard,
    # but FileHandler uses 'a' by default. Let's try 'a' with a smaller buffer if possible,
    # or just force flushing. Manual flushing is more reliable for immediate write.
    # We will add manual flushing calls after this setup.
    file_handler = logging.FileHandler(log_file_path, mode='w') # Use 'w' to overwrite each run or 'a' to append
    file_handler.setLevel(LOG_LEVEL)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(module)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    # Stream Handler for console output
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(LOG_LEVEL)
    stream_handler.setFormatter(formatter) # Use the same formatter
    logger.addHandler(stream_handler)

    logger.info(f"Logging initialized. Logs will be saved to {log_file_path}")

except Exception as e:
    # Fallback: print an error if logging setup fails
    print(f"Error setting up logging handlers: {e}", flush=True)
    # Revert to basicConfig just for console output if file logging failed
    logging.basicConfig(level=LOG_LEVEL, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')
    logger = logging.getLogger(__name__) # Get the root logger now configured by basicConfig
    logger.warning("File logging setup failed, falling back to console-only logging.")
    logger.info(f"Logging initialized (console only). Failed to save to {log_file_path}")




# Create a folder to download dataset if it does not exists
if not os.path.exists(BINARY_DATA_ROOT):
    os.makedirs(BINARY_DATA_ROOT, exist_ok=True)
    logger.warning(f"Warning: Data directory '{BINARY_DATA_ROOT}' was not found and has been created.")
    logger.info(f"Please ensure QuickDraw .bin files (e.g., full_binary_apple.bin) for categories {QUICKDRAW_CATEGORIES} are placed there.")

# Create directory for saving models if it doesn't exist
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    logger.info(f"Created directory for saving models: {MODEL_SAVE_PATH}")

2025-05-29 09:05:51,487 - INFO - <ipython-input-3-9aba89d13c7b> - Logging initialized. Logs will be saved to ./logs/quickdraw_benchmark_20250529_090551_cuda.log
INFO:__main__:Logging initialized. Logs will be saved to ./logs/quickdraw_benchmark_20250529_090551_cuda.log
2025-05-29 09:05:51,490 - WARNING - <ipython-input-3-9aba89d13c7b> - Warning: Data directory './data' was not found and has been created.
2025-05-29 09:05:51,492 - INFO - <ipython-input-3-9aba89d13c7b> - Please ensure QuickDraw .bin files (e.g., full_binary_apple.bin) for categories ['apple', 'cat', 'dog', 'door', 'elephant', 'fish', 'flower', 'grapes', 'grass', 'house', 'ice cream', 'jail', 'key', 'lion', 'moon', 'nose', 'pencil', 'rabbit', 'sun', 'tree', 'umbrella', 'van', 'cake', 'airplane', 'ant', 'banana', 'bed', 'bee', 'bicycle', 'bird', 'book', 'bread', 'bus', 'elbow', 'ear', 'camera', 'car', 'chair', 'clock', 'cloud', 'hand', 'computer', 'cookie', 'cow', 'crayon', 'cup', 'eraser', 'carrot', 'drums', 'eye', 'knife

In [ ]:

# --- Part 1: Data Download and Preparation ---
# %%
def download_quickdraw_binary(category_name, download_dir):
    """
    Downloads the .bin file for a given QuickDraw category.
    Files are named 'full_binary_{category_name_underscored}.bin'.
    """
    # Sanitize category name for filename (replace spaces with underscores)
    filename_category_part = category_name.replace(' ', '_')
    local_filename = f"full_binary_{filename_category_part}.bin"
    local_filepath = os.path.join(download_dir, local_filename)

    if os.path.exists(local_filepath):
        logger.info(f"File for '{category_name}' already exists: {local_filepath}")
        return

    # URL encode category name for the download URL (e.g., "ice cream" -> "ice%20cream")
    url_category_part = urllib.parse.quote(category_name)
    url = f"https://storage.googleapis.com/quickdraw_dataset/full/binary/{url_category_part}.bin"

    logger.info(f"Downloading '{category_name}' from {url} to {local_filepath}...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors

        total_size = int(response.headers.get('content-length', 0))

        with open(local_filepath, 'wb') as f, tqdm(
            desc=category_name,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in response.iter_content(chunk_size=8192):
                size = f.write(chunk)
                bar.update(size)
        logger.info(f"Successfully downloaded '{category_name}'.")
    except requests.exceptions.RequestException as e:
        logger.info(f"Error downloading '{category_name}': {e}")
        if os.path.exists(local_filepath): # Clean up partial download
            os.remove(local_filepath)
    except Exception as e:
        logger.info(f"An unexpected error occurred while downloading '{category_name}': {e}")
        if os.path.exists(local_filepath): # Clean up partial download
            os.remove(local_filepath)


logger.info(f"Starting download process for {len(QUICKDRAW_CATEGORIES)} categories into '{BINARY_DATA_ROOT}'...")
for category in QUICKDRAW_CATEGORIES:
    download_quickdraw_binary(category, BINARY_DATA_ROOT)
logger.info("Download process finished.")

# Explicitly flush handlers after a significant phase
for handler in logger.handlers:
    if isinstance(handler, logging.FileHandler):
        handler.flush()



2025-05-29 09:05:51,507 - INFO - <ipython-input-4-5b3215ba0243> - Starting download process for 51 categories into './data'...
INFO:__main__:Starting download process for 51 categories into './data'...
2025-05-29 09:05:51,509 - INFO - <ipython-input-4-5b3215ba0243> - Downloading 'apple' from https://storage.googleapis.com/quickdraw_dataset/full/binary/apple.bin to ./data/full_binary_apple.bin...
INFO:__main__:Downloading 'apple' from https://storage.googleapis.com/quickdraw_dataset/full/binary/apple.bin to ./data/full_binary_apple.bin...


apple:   0%|          | 0.00/13.2M [00:00<?, ?iB/s]

2025-05-29 09:05:52,090 - INFO - <ipython-input-4-5b3215ba0243> - Successfully downloaded 'apple'.
INFO:__main__:Successfully downloaded 'apple'.
2025-05-29 09:05:52,094 - INFO - <ipython-input-4-5b3215ba0243> - Downloading 'cat' from https://storage.googleapis.com/quickdraw_dataset/full/binary/cat.bin to ./data/full_binary_cat.bin...
INFO:__main__:Downloading 'cat' from https://storage.googleapis.com/quickdraw_dataset/full/binary/cat.bin to ./data/full_binary_cat.bin...


cat:   0%|          | 0.00/18.7M [00:00<?, ?iB/s]

2025-05-29 09:05:52,677 - INFO - <ipython-input-4-5b3215ba0243> - Successfully downloaded 'cat'.
INFO:__main__:Successfully downloaded 'cat'.
2025-05-29 09:05:52,681 - INFO - <ipython-input-4-5b3215ba0243> - Downloading 'dog' from https://storage.googleapis.com/quickdraw_dataset/full/binary/dog.bin to ./data/full_binary_dog.bin...
INFO:__main__:Downloading 'dog' from https://storage.googleapis.com/quickdraw_dataset/full/binary/dog.bin to ./data/full_binary_dog.bin...


...

2025-05-29 09:06:16,015 - INFO - <ipython-input-4-5b3215ba0243> - Successfully downloaded 'drums'.
INFO:__main__:Successfully downloaded 'drums'.
2025-05-29 09:06:16,018 - INFO - <ipython-input-4-5b3215ba0243> - Downloading 'eye' from https://storage.googleapis.com/quickdraw_dataset/full/binary/eye.bin to ./data/full_binary_eye.bin...
INFO:__main__:Downloading 'eye' from https://storage.googleapis.com/quickdraw_dataset/full/binary/eye.bin to ./data/full_binary_eye.bin...


eye:   0%|          | 0.00/16.1M [00:00<?, ?iB/s]

2025-05-29 09:06:16,646 - INFO - <ipython-input-4-5b3215ba0243> - Successfully downloaded 'eye'.
INFO:__main__:Successfully downloaded 'eye'.
2025-05-29 09:06:16,650 - INFO - <ipython-input-4-5b3215ba0243> - Downloading 'knife' from https://storage.googleapis.com/quickdraw_dataset/full/binary/knife.bin to ./data/full_binary_knife.bin...
INFO:__main__:Downloading 'knife' from https://storage.googleapis.com/quickdraw_dataset/full/binary/knife.bin to ./data/full_binary_knife.bin...


knife:   0%|          | 0.00/12.0M [00:00<?, ?iB/s]

2025-05-29 09:06:17,059 - INFO - <ipython-input-4-5b3215ba0243> - Successfully downloaded 'knife'.
INFO:__main__:Successfully downloaded 'knife'.
2025-05-29 09:06:17,061 - INFO - <ipython-input-4-5b3215ba0243> - Download process finished.
INFO:__main__:Download process finished.


# QuickDrawBinaryDataset Loader and Indexer

In [ ]:
# --- QuickDraw Binary Data Reading Functions (from user - Unchanged) ---
def unpack_drawing(file_handle):
    try:
        key_id, = unpack('Q', file_handle.read(8))
        country_code, = unpack('2s', file_handle.read(2))
        recognized, = unpack('b', file_handle.read(1))
        timestamp, = unpack('I', file_handle.read(4))
        n_strokes, = unpack('H', file_handle.read(2))
        image_strokes = []
        for _ in range(n_strokes):
            n_points, = unpack('H', file_handle.read(2))
            fmt = str(n_points) + 'B'
            if n_points == 0:
                image_strokes.append((tuple(), tuple()))
                continue

            x_bytes = file_handle.read(n_points)
            y_bytes = file_handle.read(n_points)

            if len(x_bytes) < n_points or len(y_bytes) < n_points:
                logger.error(f"Insufficient data for stroke points. Expected {n_points}, got {len(x_bytes)} for x, {len(y_bytes)} for y. Skipping drawing.")
                raise struct.error("Insufficient data for stroke points, likely corrupted drawing record.")

            x = unpack(fmt, x_bytes)
            y = unpack(fmt, y_bytes)
            image_strokes.append((x, y))

        return {
            'key_id': key_id,
            'country_code': country_code,
            'recognized': recognized,
            'timestamp': timestamp,
            'image': image_strokes
        }
    except struct.error as e:
        logger.debug(f"Struct error during unpack_drawing: {e}. File pointer at {file_handle.tell() if hasattr(file_handle, 'tell') else 'N/A'}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error in unpack_drawing: {e}")
        raise


def unpack_drawings(filename): # Unchanged, used by indexing if needed elsewhere
    file_size = os.path.getsize(filename)
    with open(filename, 'rb') as f, tqdm(total=file_size, unit='B', unit_scale=True, desc=f"Unpacking {os.path.basename(filename)}", leave=False) as pbar:
        while True:
            try:
                start_pos = f.tell()
                if start_pos >= file_size:
                    break
                yield unpack_drawing(f)
                pbar.update(f.tell() - start_pos)
            except struct.error:
                logger.debug(f"Struct.error in unpack_drawings, likely end of file or data. File: {filename}")
                break
            except EOFError:
                logger.debug(f"EOFError in unpack_drawings. File: {filename}")
                break

# --- Custom QuickDraw Dataset from Local Binary Files (REFACTORED) ---
class QuickDrawBinaryDataset(Dataset):
    IMAGE_SIZE = (256, 256)
    LINE_WIDTH = 2
    # Static dictionary to store cached indices for each file path
    _cached_drawing_offsets = {}

    def __init__(self, root, category, transform=None, cache_size=QUICKDRAW_CACHE_SIZE):
        self.root = root
        # Sanitize category name for filename (replace spaces with underscores)
        self.category = category.replace(' ', '_')
        self.transform = transform
        self.filepath = os.path.join(self.root, f"full_binary_{self.category}.bin")

        self.cache_size = cache_size
        self.drawing_cache = collections.OrderedDict() # LRU cache for raw drawing data

        if not os.path.exists(self.filepath):
            raise FileNotFoundError(
                f"Dataset binary file not found: {self.filepath}. Please ensure it exists."
            )

        # Load or create the index for this file path
        self.drawing_offsets = self._get_or_create_index()

        if not self.drawing_offsets:
            logger.warning(f"No drawings were indexed for category {self.category} from {self.filepath}.")
        else:
            logger.info(f"Successfully loaded or indexed {len(self.drawing_offsets)} drawings for {self.category}. Cache capacity: {self.cache_size} items.")

    def _get_or_create_index(self):
        """Gets the drawing offsets from cache or indexes the file if not cached."""
        if self.filepath in self._cached_drawing_offsets:
            logger.debug(f"Using cached index for {self.filepath}")
            return self._cached_drawing_offsets[self.filepath]

        logger.info(f"Indexing drawings from {self.filepath} for category {self.category}...")
        drawing_offsets = []

        file_size = os.path.getsize(self.filepath)
        idx_file_handle = None
        try:
            idx_file_handle = open(self.filepath, 'rb')
            with tqdm(total=file_size, unit='B', unit_scale=True, desc=f"Indexing {os.path.basename(self.filepath)}", leave=False) as pbar:
                while True:
                    current_pos = idx_file_handle.tell()
                    if current_pos >= file_size:
                        break
                    try:
                        # Read one drawing just to advance the pointer and validate structure
                        # We use the external unpack_drawing function here
                        _ = unpack_drawing(idx_file_handle)
                        drawing_offsets.append(current_pos)
                        pbar.update(idx_file_handle.tell() - current_pos)
                    except struct.error:
                        logger.debug(f"Finished indexing or encountered struct.error at offset {current_pos} in {self.filepath}. Total indexed: {len(drawing_offsets)}")
                        break
                    except EOFError:
                        logger.debug(f"EOFError encountered while indexing {self.filepath} at offset {current_pos}. Total indexed: {len(drawing_offsets)}")
                        break
                    except Exception as e:
                        logger.error(f"Unexpected error during indexing of {self.filepath} at offset {current_pos}: {e}. Stopping indexing for this file.")
                        break
        finally:
            if idx_file_handle:
                idx_file_handle.close()

        # Store the index in the cache
        self._cached_drawing_offsets[self.filepath] = drawing_offsets
        return drawing_offsets

    def _render_drawing_to_image(self, drawing_strokes):
        image = Image.new("L", self.IMAGE_SIZE, "white")
        draw = ImageDraw.Draw(image)
        for stroke_x, stroke_y in drawing_strokes:
            if not stroke_x or not stroke_y:
                continue
            if len(stroke_x) == 1:
                draw.point((int(stroke_x[0]), int(stroke_y[0])), fill="black")
            else:
                points = list(zip(stroke_x, stroke_y))
                draw.line(points, fill="black", width=self.LINE_WIDTH)
        return image

    def __len__(self):
        return len(self.drawing_offsets)

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self.drawing_offsets):
            raise IndexError(f"Index {idx} out of bounds for {len(self.drawing_offsets)} drawings.")

        drawing_data = None
        if idx in self.drawing_cache:
            drawing_data = self.drawing_cache[idx]
            self.drawing_cache.move_to_end(idx) # Mark as recently used
            # logger.debug(f"Cache hit for idx {idx} in {self.category}")
        else:
            # logger.debug(f"Cache miss for idx {idx} in {self.category}. Loading from disk.")
            offset = self.drawing_offsets[idx]
            try:
                # Open file, seek, read one drawing, then close.
                # This is safer for multiprocessing in DataLoader.
                with open(self.filepath, 'rb') as f:
                    f.seek(offset)
                    # Use the external unpack_drawing function here
                    drawing_data = unpack_drawing(f)
            except Exception as e:
                logger.error(f"Error reading drawing at index {idx}, offset {offset} from {self.filepath}: {e}")
                raise IOError(f"Failed to load drawing {idx} for {self.category}") from e

            if self.cache_size > 0:
                self.drawing_cache[idx] = drawing_data
                if len(self.drawing_cache) > self.cache_size:
                    self.drawing_cache.popitem(last=False) # Remove oldest item (LRU)

        if drawing_data is None: # Should not happen if logic is correct
             raise RuntimeError(f"Drawing data for index {idx} could not be retrieved.")

        pil_image = self._render_drawing_to_image(drawing_data['image'])

        if self.transform:
            pil_image = self.transform(pil_image)

        return pil_image, self.category

In [ ]:
# --- Data Preparation with Augmentation ---
def get_augmented_quickdraw_data(categories, num_train_per_cat, num_test_per_cat, data_root):
    """"
    Enhanced version of get_quickdraw_data that creates train/validation/test splits
    and applies data augmentation to the training set.
    """
    logger.info(f"Loading augmented QuickDraw data for {len(categories)} categories...")

    # Create standard transforms
    base_transform = T.Compose([
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Augmented transform for training
    train_transform = T.Compose([
        T.Grayscale(num_output_channels=3),
        T.RandomHorizontalFlip(p=0.5),
        T.RandomRotation(15),
        T.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        T.ColorJitter(brightness=0.2, contrast=0.2),
        base_transform
    ])

    # Simple transform for validation/testing
    test_transform = T.Compose([
        T.Grayscale(num_output_channels=3),
        base_transform
    ])

    all_train_datasets = []
    all_val_datasets = []
    all_test_datasets = []

    for category_idx, category_name in enumerate(categories):
        try:
            logger.info(f"Loading QuickDraw category: {category_name} from local binary files...")
            full_category_dataset = QuickDrawBinaryDataset(
                root=data_root,
                category=category_name,
                transform=None,  # No transform yet, will be applied per-split
                cache_size=QUICKDRAW_CACHE_SIZE
            )

            if len(full_category_dataset) == 0:
                logger.warning(f"No data loaded for category {category_name}. Skipping.")
                continue

            # Calculate actual samples to use (limited by available data)
            actual_num_train = min(num_train_per_cat, len(full_category_dataset) * 0.7)
            remaining_samples = len(full_category_dataset) - actual_num_train

            # Split remaining samples between validation and test
            actual_num_val = min(num_train_per_cat * 0.2, remaining_samples * 0.5)
            actual_num_test = min(num_test_per_cat, remaining_samples - actual_num_val)

            actual_num_train = int(actual_num_train)
            actual_num_val = int(actual_num_val)
            actual_num_test = int(actual_num_test)

            logger.info(f"Category {category_name}: {actual_num_train} train, {actual_num_val} val, {actual_num_test} test")

            # Ensure we have enough samples
            if actual_num_train == 0 or actual_num_val == 0 or actual_num_test == 0:
                logger.warning(f"Not enough samples in {category_name} for desired splits. Skipping.")
                continue

            # Create indices for random splits
            indices = list(range(len(full_category_dataset)))
            random.shuffle(indices)

            train_indices = indices[:actual_num_train]
            val_indices = indices[actual_num_train:actual_num_train + actual_num_val]
            test_indices = indices[actual_num_train + actual_num_val:actual_num_train + actual_num_val + actual_num_test]

            # Create labeled datasets with appropriate transforms
            train_subset = TransformedSubset(full_category_dataset, train_indices, train_transform, category_idx)
            val_subset = TransformedSubset(full_category_dataset, val_indices, test_transform, category_idx)
            test_subset = TransformedSubset(full_category_dataset, test_indices, test_transform, category_idx)

            all_train_datasets.append(train_subset)
            all_val_datasets.append(val_subset)
            all_test_datasets.append(test_subset)

        except Exception as e:
            logger.error(f"Error loading category {category_name}: {e}")
            continue

    # Ensure we have data for at least some categories
    if not all_train_datasets or not all_val_datasets or not all_test_datasets:
        if not any(all_train_datasets) and not any(all_val_datasets) and not any(all_test_datasets):
            raise RuntimeError("No QuickDraw data could be loaded for any category. Aborting.")
        else:
            logger.warning("Some categories failed to load, proceeding with available data.")

    # Combine datasets across categories
    train_dataset = ConcatDataset(all_train_datasets)
    val_dataset = ConcatDataset(all_val_datasets)
    test_dataset = ConcatDataset(all_test_datasets)

    logger.info(f"Created datasets with {len(train_dataset)} training samples, "
               f"{len(val_dataset)} validation samples, and {len(test_dataset)} test samples.")

    return train_dataset, val_dataset, test_dataset

# Helper class for applying transforms during subset creation
class TransformedSubset(Dataset):
    """
    A subset of a dataset with a transform that's applied on-the-fly.
    Also assigns a fixed class label for classification tasks.
    """
    def __init__(self, dataset, indices, transform, label):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform
        self.label = label

    def __getitem__(self, idx):
        image, _ = self.dataset[self.indices[idx]]
        if self.transform:
            image = self.transform(image)
        return image, self.label

    def __len__(self):
        return len(self.indices)

# Model Classes

In [ ]:

class MobileNetV3FeatureExtractor(torch.nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.features = original_model.features
        self.avgpool = original_model.avgpool
        self.flatten = torch.nn.Flatten(1)
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return x

class ShuffleNetV2FeatureExtractor(torch.nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.conv1 = original_model.conv1
        self.maxpool = original_model.maxpool
        self.stage2 = original_model.stage2
        self.stage3 = original_model.stage3
        self.stage4 = original_model.stage4
        self.conv5 = original_model.conv5
        self.glob_pool = torch.nn.AdaptiveAvgPool2d((1,1))
        self.flatten = torch.nn.Flatten(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.conv5(x)
        x = self.glob_pool(x)
        x = self.flatten(x)
        return x

class SqueezeNetFeatureExtractor(torch.nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.features = original_model.features
        self.pool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = torch.nn.Flatten(1)
    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.flatten(x)
        return x

class EfficientNetFeatureExtractor(torch.nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.features = original_model.features
        self.avgpool = original_model.avgpool
        self.flatten = torch.nn.Flatten(1)
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        return x


Extract Features

In [ ]:
# --- Feature Extraction (MODIFIED) ---
def extract_features(model, dataloader, device, description="Extracting features"):
    model.eval()
    model.to(device)
    features_list = []
    labels_list = []
    for inputs, labels in tqdm(dataloader, desc=description, leave=False, unit="batch"):
        inputs = inputs.to(device)
        with torch.no_grad(): # Ensure no gradients are computed in this block
            outputs = model(inputs)
        features_list.append(outputs.cpu().detach().numpy()) # Detach before converting to numpy
        # Labels are typically not part of the computation graph, but detaching is harmless
        if isinstance(labels, torch.Tensor):
            labels_list.append(labels.cpu().detach().numpy())
        else: # If labels are already numpy arrays or other types
            labels_list.append(labels)

    if not features_list:
        return np.array([]), np.array([])

    return np.concatenate(features_list), np.concatenate(labels_list)


# Benchmarks

### Benchmark Lightweight models for Feature Extraction

In [ ]:

# --- Main Benchmarking Loop (MODIFIED calls to extract_features) ---
def run_feat_ext_benchmark():
    results = []
    logger.info(f"Using device: {DEVICE}")
    logger.info(f"Benchmarking on QuickDraw categories: {', '.join(QUICKDRAW_CATEGORIES)}")
    logger.info(f"Samples per category: {NUM_TRAIN_SAMPLES_PER_CATEGORY} train, {NUM_TEST_SAMPLES_PER_CATEGORY} test.")
    logger.info(f"Loading QuickDraw data from local directory: {os.path.abspath(BINARY_DATA_ROOT)}\\n")


    for model_name, config in tqdm(MODELS_TO_TEST.items(), desc="Benchmarking Models (Feature Extraction)", unit="model"):
        logger.info(f"--- Testing Model (Feature Extraction): {model_name} ---")

        current_model_accuracy = "Error"
        current_model_acc_per_param = "Error"
        current_model_feat_ext_time = "N/A"
        current_model_train_time = "N/A"
        current_model_lr_inference_time = "N/A"
        current_model_lr_inference_speed = "N/A"
        model_params = "Error" # Initialize in case model loading fails

        try:
            weights = config["weights"]
            base_model = config["model_fn"](weights=weights)
            model_params = sum(p.numel() for p in base_model.parameters() if p.requires_grad) / 1_000_000

            model_specific_transforms = weights.transforms()

            train_dataset, test_dataset = get_augmented_quickdraw_data(
                QUICKDRAW_CATEGORIES,
                NUM_TRAIN_SAMPLES_PER_CATEGORY,
                NUM_TEST_SAMPLES_PER_CATEGORY,
                BINARY_DATA_ROOT
            )

            if len(train_dataset) == 0 or len(test_dataset) == 0:
                logger.warning(f"Not enough data loaded for {model_name} to proceed. Skipping.")
                raise RuntimeError("Empty dataset after loading.")

            effective_batch_size = get_adaptive_batch_size(NUM_TRAIN_SAMPLES_PER_CATEGORY, len(QUICKDRAW_CATEGORIES))
            logger.info(f"Using adaptive batch size of {effective_batch_size} for feature extraction")

            train_loader = DataLoader(train_dataset, batch_size=effective_batch_size, shuffle=False) # Shuffle False for feature extraction is fine
            test_loader = DataLoader(test_dataset, batch_size=effective_batch_size, shuffle=False)

            feature_extractor = config["feature_extractor_fn"](base_model)
            feature_extractor.to(DEVICE)
            feature_extractor.eval()

            logger.info("Extracting training features...")
            start_time = time.time()
            train_features, train_labels = extract_features(feature_extractor, train_loader, DEVICE, description=f"Extracting train features ({model_name})")
            feat_ext_time_train = time.time() - start_time

            if train_features.size == 0:
                logger.warning(f"No training features extracted for {model_name}. Skipping further steps for this model.")
                raise RuntimeError("No training features.")
            logger.debug(f"Train features shape: {train_features.shape}, Train labels shape: {train_labels.shape}")

            logger.info("Extracting test features...")
            start_time = time.time()
            test_features, test_labels = extract_features(feature_extractor, test_loader, DEVICE, description=f"Extracting test features ({model_name})")
            feat_ext_time_test = time.time() - start_time

            if test_features.size == 0:
                logger.warning(f"No test features extracted for {model_name}. Skipping further steps for this model.")
                raise RuntimeError("No test features.")
            logger.debug(f"Test features shape: {test_features.shape}, Test labels shape: {test_labels.shape}")

            current_model_feat_ext_time = f"{feat_ext_time_train + feat_ext_time_test:.2f}"

            logger.info("Training Logistic Regression classifier...")
            start_time = time.time()
            classifier = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42, solver='liblinear', C=0.1))
            classifier.fit(train_features, train_labels)
            current_model_train_time = f"{time.time() - start_time:.2f}"

            accuracy = classifier.score(test_features, test_labels) * 100
            logger.info(f"QuickDraw Test Accuracy for {model_name} (LogReg): {accuracy:.2f}%")
            current_model_accuracy = f"{accuracy:.2f}"
            current_model_acc_per_param = f"{accuracy / model_params if isinstance(model_params, (int, float)) and model_params > 0 else 0:.2f}"

            # Calculate Logistic Regression Inference Time and Speed
            start_time_lr_inference = time.time()
            _ = classifier.predict(test_features) # Perform prediction
            lr_inference_duration = time.time() - start_time_lr_inference
            current_model_lr_inference_time = f"{lr_inference_duration:.4f}" # Higher precision for inference time
            num_test_samples_lr = len(test_labels)
            current_model_lr_inference_speed = f"{num_test_samples_lr / lr_inference_duration if lr_inference_duration > 0 else 0:.2f}"
            logger.info(f"Logistic Regression Inference Time: {current_model_lr_inference_time}s, Speed: {current_model_lr_inference_speed} samples/s")


        except RuntimeError as e:
            logger.error(f"Runtime error during feature extraction processing for {model_name}: {e}")
        except Exception as e:
            logger.exception(f"An unexpected error occurred for model {model_name} (Feature Extraction): {e}") # Use logger.exception for traceback
        finally:
            results.append({
                "Model": model_name,
                "Accuracy (%)": current_model_accuracy,
                "Params (M)": f"{model_params:.2f}" if isinstance(model_params, (int, float)) else model_params,
                "Acc/Params": current_model_acc_per_param,
                "Feat Ext Time (s)": current_model_feat_ext_time,
                "LR Train Time (s)": current_model_train_time,
                "LR Inference Time (s)": current_model_lr_inference_time,
                "LR Inference Speed (samples/s)": current_model_lr_inference_speed
            })
            logger.info("-" * 30 + "\\n")
            if 'base_model' in locals(): del base_model
            if 'feature_extractor' in locals(): del feature_extractor
            if 'train_dataset' in locals(): del train_dataset
            if 'test_dataset' in locals(): del test_dataset
            if 'train_loader' in locals(): del train_loader
            if 'test_loader' in locals(): del test_loader
            if 'train_features' in locals(): del train_features
            if 'train_labels' in locals(): del train_labels
            if 'test_features' in locals(): del test_features
            if 'test_labels' in locals(): del test_labels
            if DEVICE == torch.device("cuda"):
                torch.cuda.empty_cache()

    logger.info("\\n--- Feature Extraction Benchmark Results ---")
    for handler in logger.handlers:
        if isinstance(handler, logging.FileHandler):
            handler.flush()
    if not results:
        logger.info("No results to display for feature extraction benchmark.")
        return

    headers = results[0].keys()
    col_widths = {key: len(key) for key in headers}
    for row in results:
        for key in headers:
            col_widths[key] = max(col_widths.get(key, 0), len(str(row.get(key, ""))))

    for key in col_widths:
        col_widths[key] += 2

    header_line = " | ".join(f"{h:<{col_widths[h]}}" for h in headers)
    logger.info(header_line)
    logger.info("-" * len(header_line))

    for row in results:
        row_line = " | ".join(f"{str(row.get(h, 'N/A')):<{col_widths[h]}}" for h in headers)
        logger.info(row_line)

    if results:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_filename = f"feature_extraction_benchmark_results_{timestamp}.csv"
        try:
            with open(results_filename, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=headers)
                writer.writeheader()
                writer.writerows(results)
            logger.info(f"Feature extraction benchmark results saved to {results_filename}")
        except IOError as e:
            logger.error(f"Could not save feature extraction results to CSV: {e}")

### Benchmark Lightweight models for Fine-tuning

In [ ]:
# --- Enhanced Fine-tuning Benchmarking Loop (Part 1) ---
import copy
import random
import torch.optim as optim
import torch.nn as nn

def run_finetuning_benchmark():
    finetuning_results = []

    # Get data with augmentation
    train_dataset, val_dataset, test_dataset = get_augmented_quickdraw_data(
        QUICKDRAW_CATEGORIES,
        NUM_TRAIN_SAMPLES_PER_CATEGORY,
        NUM_TEST_SAMPLES_PER_CATEGORY,
        BINARY_DATA_ROOT
    )

    # In run_finetuning_benchmark function:
    # Calculate adaptive batch size based on dataset size
    effective_batch_size = get_adaptive_batch_size(NUM_TRAIN_SAMPLES_PER_CATEGORY, len(QUICKDRAW_CATEGORIES))
    logger.info(f"Using adaptive batch size of {effective_batch_size} for feature extraction")

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=effective_batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=effective_batch_size, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=effective_batch_size, shuffle=False, num_workers=2, pin_memory=True)

    logger.info(f"\n--- Starting Enhanced Fine-tuning Benchmark ---")
    logger.info(f"Using device: {DEVICE}")
    logger.info(f"Fine-tuning on QuickDraw categories: {', '.join(QUICKDRAW_CATEGORIES)}")
    logger.info(f"Samples per category: {NUM_TRAIN_SAMPLES_PER_CATEGORY} train, {NUM_TEST_SAMPLES_PER_CATEGORY} test.")
    logger.info(f"Number of fine-tuning epochs: {NUM_FINETUNE_EPOCHS}, Learning Rate: {FINETUNE_LEARNING_RATE}")
    logger.info(f"Weight Decay: {FINETUNE_WEIGHT_DECAY}, Gradual Unfreezing: {USE_GRADUAL_UNFREEZING}")
    logger.info(f"Loading QuickDraw data from local directory: {os.path.abspath(BINARY_DATA_ROOT)}\n")

    num_classes = len(QUICKDRAW_CATEGORIES)

    for model_name, config in tqdm(MODELS_TO_TEST.items(), desc="Benchmarking Models (Enhanced Fine-tuning)", unit="model"):
        logger.info(f"--- Enhanced Fine-tuning Model: {model_name} ---")

        current_ft_accuracy = "Error"
        current_ft_train_time = "N/A"
        current_ft_inference_time = "N/A"
        current_ft_inference_speed = "N/A"
        model_params_ft = "Error"
        current_model_path = "Not saved"
        best_val_accuracy = 0.0

        try:
            weights = config["weights"]
            model_to_finetune = config["model_fn"](weights=weights)
            model_params_ft = sum(p.numel() for p in model_to_finetune.parameters() if p.requires_grad) / 1_000_000

            # Create custom classifier heads with improved architectures
            if model_name == "MobileNetV3-Small":
                # Save the feature extraction part
                features = model_to_finetune.features
                avgpool = model_to_finetune.avgpool

                # Replace classifier with a better one
                in_features = model_to_finetune.classifier[0].in_features
                model_to_finetune.classifier = nn.Sequential(
                    nn.Linear(in_features, 1024),
                    nn.BatchNorm1d(1024),
                    nn.ReLU(inplace=True),
                    nn.Dropout(0.4),
                    nn.Linear(1024, 512),
                    nn.BatchNorm1d(512),
                    nn.ReLU(inplace=True),
                    nn.Dropout(0.3),
                    nn.Linear(512, num_classes)
                )

                # Define the feature layers for gradual unfreezing
                feature_layers = [features[0], features[1], features[2], features[3],
                                 features[4], features[5], features[6], features[7],
                                 features[8], features[9], features[10], features[11],
                                 avgpool]

            elif model_name == "SqueezeNet1_1":
                # Get the input channels of the classifier
                in_channels = model_to_finetune.classifier[1].in_channels

                # More sophisticated conv classifier
                model_to_finetune.classifier = nn.Sequential(
                    nn.Dropout(p=0.5),
                    nn.Conv2d(in_channels, 512, kernel_size=1),
                    nn.ReLU(inplace=True),
                    nn.AdaptiveAvgPool2d((1, 1)),
                    nn.Conv2d(512, num_classes, kernel_size=1)
                )

                # Define feature layers for gradual unfreezing
                feature_layers = [
                    model_to_finetune.features[0],  # conv1
                    model_to_finetune.features[1],  # maxpool
                    model_to_finetune.features[2],  # fire1
                    model_to_finetune.features[3],  # fire2
                    model_to_finetune.features[4],  # fire3
                    model_to_finetune.features[5],  # fire4
                    model_to_finetune.features[6],  # maxpool
                    model_to_finetune.features[7],  # fire5
                    model_to_finetune.features[8],  # fire6
                    model_to_finetune.features[9],  # fire7
                    model_to_finetune.features[10], # fire8
                    model_to_finetune.features[11], # maxpool
                    model_to_finetune.features[12]  # conv10
                ]
            else:
                logger.warning(f"Classifier modification not defined for {model_name}. Skipping fine-tuning.")
                continue

            model_to_finetune.to(DEVICE)

            # --- Enhanced Fine-tuning Procedure ---
            logger.info(f"Starting fine-tuning for {model_name}...")
            start_time = time.time()
            best_model_wts = copy.deepcopy(model_to_finetune.state_dict())

            # Initialize optimizer, criterion, and scheduler
            optimizer = optim.Adam(model_to_finetune.parameters(), lr=FINETUNE_LEARNING_RATE, weight_decay=FINETUNE_WEIGHT_DECAY)
            criterion = nn.CrossEntropyLoss()
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True)

            # Initialize early stopping variables
            early_stopping_patience = 3
            early_stopping_counter = 0

            # Fine-tuning loop
            for epoch in range(NUM_FINETUNE_EPOCHS):
                logger.info(f"Epoch {epoch + 1}/{NUM_FINETUNE_EPOCHS}")

                # Adjust learning rate if using gradual unfreezing
                if USE_GRADUAL_UNFREEZING:
                    if epoch < 10:
                        for param in feature_layers[:epoch + 1]:
                            for p in param.parameters():
                                p.requires_grad = True
                    else:
                        for param in feature_layers:
                            for p in param.parameters():
                                p.requires_grad = True

                # Training phase
                model_to_finetune.train()
                train_loss = 0.0
                train_correct = 0
                train_total = 0
                
                # Calculate needed gradient accumulation steps based on effective batch size
                if effective_batch_size < MAX_BATCH_SIZE:
                    gradient_accumulation_steps = max(1, MAX_BATCH_SIZE // effective_batch_size)
                    logger.info(f"Using gradient accumulation with {gradient_accumulation_steps} steps")
                else:
                    gradient_accumulation_steps = 1
                
                # Zero gradients at the beginning of epoch
                optimizer.zero_grad()
                
                for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
                    images, labels = images.to(DEVICE), labels.to(DEVICE)

                    # Forward pass
                    outputs = model_to_finetune(images)
                    loss = criterion(outputs, labels) / gradient_accumulation_steps  # Normalize loss

                    # Backward pass
                    loss.backward()

                    # Only update weights after accumulating gradients for specified steps
                    if (batch_idx + 1) % gradient_accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                        optimizer.step()
                        optimizer.zero_grad()

                    train_loss += loss.item() * images.size(0) * gradient_accumulation_steps  # Scale loss back for reporting
                    _, predicted = torch.max(outputs, 1)
                    train_correct += (predicted == labels).sum().item()
                    train_total += labels.size(0)

                # Calculate average loss and accuracy for the epoch
                epoch_train_loss = train_loss / train_total
                epoch_train_accuracy = 100 * train_correct / train_total
                logger.info(f"Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {epoch_train_accuracy:.2f}%")

                # Validation phase
                model_to_finetune.eval()
                val_loss = 0.0
                val_correct = 0
                val_total = 0

                with torch.no_grad():
                    for images, labels in tqdm(val_loader, desc="Validation", leave=False):
                        images, labels = images.to(DEVICE), labels.to(DEVICE)

                        # Forward pass
                        outputs = model_to_finetune(images)
                        loss = criterion(outputs, labels)

                        val_loss += loss.item() * images.size(0)
                        _, predicted = torch.max(outputs, 1)
                        val_correct += (predicted == labels).sum().item()
                        val_total += labels.size(0)

                # Calculate average loss and accuracy for the validation set
                epoch_val_loss = val_loss / val_total
                epoch_val_accuracy = 100 * val_correct / val_total
                logger.info(f"Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_accuracy:.2f}%")

                # Early stopping logic
                if epoch_val_accuracy > best_val_accuracy:
                    best_val_accuracy = epoch_val_accuracy
                    best_model_wts = copy.deepcopy(model_to_finetune.state_dict())
                    logger.info(f"New best model found for {model_name}! (Val Accuracy: {best_val_accuracy:.2f}%)")
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
                    logger.info(f"Validation accuracy didn't improve. Counter: {early_stopping_counter}/{early_stopping_patience}")

                # Check if early stopping criteria is met
                if early_stopping_counter >= early_stopping_patience:
                    logger.info(f"Early stopping triggered after {epoch+1} epochs")
                    break

                # Learning rate scheduler
                scheduler.step(epoch_val_accuracy)
                logger.info(f"Current learning rate: {optimizer.param_groups[0]['lr']:.6f}")
                # Load the best model weights (based on validation accuracy)
                model_to_finetune.load_state_dict(best_model_wts)

                # Save the best model (highest validation accuracy)
                best_model_path = f"{MODEL_SAVE_PATH}/{model_name}_best.pth"
                torch.save(model_to_finetune.state_dict(), best_model_path)
                logger.info(f"Best model saved: {best_model_path} (Val Accuracy: {best_val_accuracy:.2f}%)")


            # Load the best model weights
            model_to_finetune.load_state_dict(best_model_wts)

            # Now save the final model (end of training)
            final_model_path = f"{MODEL_SAVE_PATH}/{model_name}_final.pth"
            # Store current state before overwriting
            final_model_wts = copy.deepcopy(model_to_finetune.state_dict())
            # Save the final state
            torch.save(final_model_wts, final_model_path)
            logger.info(f"Final model saved: {final_model_path}")


            # Evaluate the model on the test set
            test_loss = 0.0
            test_correct = 0
            test_total = 0

            # Track inference time
            start_inference_time = time.time()
            with torch.no_grad():
                for images, labels in tqdm(test_loader, desc="Testing", leave=False):
                    images, labels = images.to(DEVICE), labels.to(DEVICE)

                    # Forward pass
                    outputs = model_to_finetune(images)
                    loss = criterion(outputs, labels)

                    test_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs, 1)
                    test_correct += (predicted == labels).sum().item()
                    test_total += labels.size(0)

            inference_time = time.time() - start_inference_time
            current_ft_inference_time = f"{inference_time:.4f}"
            current_ft_inference_speed = f"{test_total / inference_time:.2f}"

            # Calculate average loss and accuracy for the test set
            test_loss /= test_total
            test_accuracy = 100 * test_correct / test_total
            logger.info(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

            # Append the results for this model to the overall results
            finetuning_results.append({
                'model_name': model_name,
                'test_loss': test_loss,
                'test_accuracy': test_accuracy,
                'params': model_params_ft,
                'train_time': current_ft_train_time,
                'inference_time': current_ft_inference_time,
                'inference_speed': current_ft_inference_speed,
                'best_val_accuracy': best_val_accuracy
            })
        except Exception as e:
            logger.error(f"Error occurred while fine-tuning {model_name}: {e}")
            finetuning_results.append({
                'model_name': model_name,
                'error': str(e)
            })

    # --- Benchmarking Results ---
    logger.info(f"\n--- Enhanced Fine-tuning Benchmarking Results ---")
    for result in finetuning_results:
        if 'error' in result:
            logger.info(f"Model: {result['model_name']}, Error: {result['error']}")
        else:
            logger.info(f"Model: {result['model_name']}, Test Loss: {result['test_loss']:.4f}, Test Accuracy: {result['test_accuracy']:.2f}%, Params: {result['params']:.2f}M, Train Time: {result['train_time']}, Inference Time: {result['inference_time']}, Inference Speed: {result['inference_speed']}")

    logger.info(f"Best model(s) based on test accuracy:")
    best_models = sorted(finetuning_results, key=lambda x: x.get('test_accuracy', 0), reverse=True)[:3]
    for bm in best_models:
        logger.info(f" - {bm['model_name']}: {bm['test_accuracy']:.2f}%")

    logger.info(f"Best model(s) based on validation accuracy:")
    best_val_models = sorted(finetuning_results, key=lambda x: x.get('best_val_accuracy', 0), reverse=True)[:3]
    for bvm in best_val_models:
        logger.info(f" - {bvm['model_name']}: {bvm['best_val_accuracy']:.2f}%")

    logger.info(f"\n--- Enhanced Fine-tuning Benchmark Completed ---")
    return finetuning_results




In [ ]:
# Run Benchmark for Feature Extraction
# run_feat_ext_benchmark()

2025-05-29 09:06:17,162 - INFO - <ipython-input-9-95701300e355> - Using device: cuda
INFO:__main__:Using device: cuda
2025-05-29 09:06:17,164 - INFO - <ipython-input-9-95701300e355> - Benchmarking on QuickDraw categories: apple, cat, dog, door, elephant, fish, flower, grapes, grass, house, ice cream, jail, key, lion, moon, nose, pencil, rabbit, sun, tree, umbrella, van, cake, airplane, ant, banana, bed, bee, bicycle, bird, book, bread, bus, elbow, ear, camera, car, chair, clock, cloud, hand, computer, cookie, cow, crayon, cup, eraser, carrot, drums, eye, knife
INFO:__main__:Benchmarking on QuickDraw categories: apple, cat, dog, door, elephant, fish, flower, grapes, grass, house, ice cream, jail, key, lion, moon, nose, pencil, rabbit, sun, tree, umbrella, van, cake, airplane, ant, banana, bed, bee, bicycle, bird, book, bread, bus, elbow, ear, camera, car, chair, clock, cloud, hand, computer, cookie, cow, crayon, cup, eraser, carrot, drums, eye, knife
2025-05-29 09:06:17,166 - INFO - <ip

Benchmarking Models (Feature Extraction):   0%|          | 0/2 [00:00<?, ?model/s]

2025-05-29 09:06:17,187 - INFO - <ipython-input-9-95701300e355> - --- Testing Model (Feature Extraction): MobileNetV3-Small ---
INFO:__main__:--- Testing Model (Feature Extraction): MobileNetV3-Small ---


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth



  0%|          | 0.00/9.83M [00:00<?, ?B/s]
 13%|█▎        | 1.25M/9.83M [00:00<00:00, 13.0MB/s]
100%|██████████| 9.83M/9.83M [00:00<00:00, 47.5MB/s]


...

2025-05-29 09:08:54,327 - INFO - <ipython-input-6-c74613b9a56b> - Loading QuickDraw category: apple from local binary files...
INFO:__main__:Loading QuickDraw category: apple from local binary files...
2025-05-29 09:08:54,329 - INFO - <ipython-input-5-51715e440719> - Successfully loaded or indexed 144722 drawings for apple. Cache capacity: 20000 items.
INFO:__main__:Successfully loaded or indexed 144722 drawings for apple. Cache capacity: 20000 items.
2025-05-29 09:08:54,330 - INFO - <ipython-input-6-c74613b9a56b> - Loaded 144722 samples for category: apple
INFO:__main__:Loaded 144722 samples for category: apple
2025-05-29 09:08:54,336 - INFO - <ipython-input-6-c74613b9a56b> - Loading QuickDraw category: cat from local binary files...
INFO:__main__:Loading QuickDraw category: cat from local binary files...
2025-05-29 09:08:54,338 - INFO - <ipython-input-5-51715e440719> - Successfully loaded or indexed 123202 drawings for cat. Cache capacity: 20000 items.
INFO:__main__:Successfully load

Extracting train features (SqueezeNet1_1):   0%|          | 0/80 [00:00<?, ?batch/s]

2025-05-29 09:09:11,687 - INFO - <ipython-input-9-95701300e355> - Extracting test features...
INFO:__main__:Extracting test features...


Extracting test features (SqueezeNet1_1):   0%|          | 0/16 [00:00<?, ?batch/s]

2025-05-29 09:09:15,092 - INFO - <ipython-input-9-95701300e355> - Training Logistic Regression classifier...
INFO:__main__:Training Logistic Regression classifier...
2025-05-29 09:10:10,170 - INFO - <ipython-input-9-95701300e355> - QuickDraw Test Accuracy for SqueezeNet1_1 (LogReg): 57.65%
INFO:__main__:QuickDraw Test Accuracy for SqueezeNet1_1 (LogReg): 57.65%
2025-05-29 09:10:10,183 - INFO - <ipython-input-9-95701300e355> - Logistic Regression Inference Time: 0.0109s, Speed: 93442.91 samples/s
INFO:__main__:Logistic Regression Inference Time: 0.0109s, Speed: 93442.91 samples/s
2025-05-29 09:10:10,185 - INFO - <ipython-input-9-95701300e355> - ------------------------------\n
INFO:__main__:------------------------------\n
2025-05-29 09:10:10,196 - INFO - <ipython-input-9-95701300e355> - \n--- Feature Extraction Benchmark Results ---
INFO:__main__:\n--- Feature Extraction Benchmark Results ---
2025-05-29 09:10:10,198 - INFO - <ipython-input-9-95701300e355> - Model               | Accura

In [ ]:
# Run Benchmark for Fine-tuning
run_finetuning_benchmark()

2025-05-29 09:10:10,215 - INFO - <ipython-input-10-622d8ba56b45> - \n--- Starting Fine-tuning Benchmark ---
INFO:__main__:\n--- Starting Fine-tuning Benchmark ---
2025-05-29 09:10:10,222 - INFO - <ipython-input-10-622d8ba56b45> - Using device: cuda
INFO:__main__:Using device: cuda
2025-05-29 09:10:10,224 - INFO - <ipython-input-10-622d8ba56b45> - Fine-tuning on QuickDraw categories: apple, cat, dog, door, elephant, fish, flower, grapes, grass, house, ice cream, jail, key, lion, moon, nose, pencil, rabbit, sun, tree, umbrella, van, cake, airplane, ant, banana, bed, bee, bicycle, bird, book, bread, bus, elbow, ear, camera, car, chair, clock, cloud, hand, computer, cookie, cow, crayon, cup, eraser, carrot, drums, eye, knife
INFO:__main__:Fine-tuning on QuickDraw categories: apple, cat, dog, door, elephant, fish, flower, grapes, grass, house, ice cream, jail, key, lion, moon, nose, pencil, rabbit, sun, tree, umbrella, van, cake, airplane, ant, banana, bed, bee, bicycle, bird, book, bread, 

Benchmarking Models (Fine-tuning):   0%|          | 0/2 [00:00<?, ?model/s]

2025-05-29 09:10:10,255 - INFO - <ipython-input-10-622d8ba56b45> - --- Fine-tuning Model: MobileNetV3-Small ---
INFO:__main__:--- Fine-tuning Model: MobileNetV3-Small ---


Processing categories:   0%|          | 0/51 [00:00<?, ?category/s]

2025-05-29 09:10:10,420 - INFO - <ipython-input-6-c74613b9a56b> - Loading QuickDraw category: apple from local binary files...
INFO:__main__:Loading QuickDraw category: apple from local binary files...
2025-05-29 09:10:10,421 - INFO - <ipython-input-5-51715e440719> - Successfully loaded or indexed 144722 drawings for apple. Cache capacity: 20000 items.
INFO:__main__:Successfully loaded or indexed 144722 drawings for apple. Cache capacity: 20000 items.
2025-05-29 09:10:10,422 - INFO - <ipython-input-6-c74613b9a56b> - Loaded 144722 samples for category: apple
INFO:__main__:Loaded 144722 samples for category: apple
2025-05-29 09:10:10,429 - INFO - <ipython-input-6-c74613b9a56b> - Loading QuickDraw category: cat from local binary files...
INFO:__main__:Loading QuickDraw category: cat from local binary files...
2025-05-29 09:10:10,431 - INFO - <ipython-input-5-51715e440719> - Successfully loaded or indexed 123202 drawings for cat. Cache capacity: 20000 items.
INFO:__main__:Successfully load

...

2025-05-29 09:11:49,832 - INFO - <ipython-input-10-622d8ba56b45> - Epoch 2/3, Train Loss: 3.5946
INFO:__main__:Epoch 2/3, Train Loss: 3.5946


Epoch 3/3 Training:   0%|          | 0/80 [00:00<?, ?batch/s]

2025-05-29 09:12:08,825 - INFO - <ipython-input-10-622d8ba56b45> - Epoch 3/3, Train Loss: 2.5741
INFO:__main__:Epoch 3/3, Train Loss: 2.5741
2025-05-29 09:12:08,828 - INFO - <ipython-input-10-622d8ba56b45> - Fine-tuning for SqueezeNet1_1 completed in 57.78s.
INFO:__main__:Fine-tuning for SqueezeNet1_1 completed in 57.78s.
2025-05-29 09:12:08,829 - INFO - <ipython-input-10-622d8ba56b45> - Evaluating fine-tuned SqueezeNet1_1...
INFO:__main__:Evaluating fine-tuned SqueezeNet1_1...


Evaluating SqueezeNet1_1:   0%|          | 0/16 [00:00<?, ?batch/s]

2025-05-29 09:12:11,734 - INFO - <ipython-input-10-622d8ba56b45> - Fine-tuned Test Accuracy for SqueezeNet1_1: 39.51%
INFO:__main__:Fine-tuned Test Accuracy for SqueezeNet1_1: 39.51%
2025-05-29 09:12:11,736 - INFO - <ipython-input-10-622d8ba56b45> - Fine-tuned Inference Time: 2.9027s, Speed: 351.39 samples/s
INFO:__main__:Fine-tuned Inference Time: 2.9027s, Speed: 351.39 samples/s
2025-05-29 09:12:11,737 - INFO - <ipython-input-10-622d8ba56b45> - ------------------------------\n
INFO:__main__:------------------------------\n
2025-05-29 09:12:11,780 - INFO - <ipython-input-10-622d8ba56b45> - \n--- Fine-tuning Benchmark Results ---
INFO:__main__:\n--- Fine-tuning Benchmark Results ---
2025-05-29 09:12:11,782 - INFO - <ipython-input-10-622d8ba56b45> - Model               | FT Accuracy (%)   | Params (M)   | FT Train Time (s)   | FT Inference Time (s)   | FT Inference Speed (samples/s)  
INFO:__main__:Model               | FT Accuracy (%)   | Params (M)   | FT Train Time (s)   | FT Inferen